In [37]:
import pandas as pd
import json

In [38]:
with open("intents .json") as intents:
  data1=json.load(intents)

In [39]:
pd.set_option('display.max_colwidth', None)

In [40]:
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
data.head()

,inputs,tags
0,Hi,greeting
1,Hey,greeting
2,Is anyone there?,greeting
3,Hi there,greeting
4,Hello,greeting


In [41]:

data

,inputs,tags
0,Hi,greeting
1,Hey,greeting
2,Is anyone there?,greeting
3,Hi there,greeting
4,Hello,greeting
...,...,...
227,How do I know if I'm unwell?,fact-29
228,How can I maintain social connections? What if I feel lonely?,fact-30
229,What's the difference between anxiety and stress?,fact-31
230,What's the difference between sadness and depression?,fact-32


In [42]:
#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Sentiment = le.fit_transform(data['tags'])

In [43]:
data["label"]=Sentiment

In [44]:
data.head()
 

,inputs,tags,label
0,Hi,greeting,44
1,Hey,greeting,44
2,Is anyone there?,greeting,44
3,Hi there,greeting,44
4,Hello,greeting,44


In [45]:
def clean(text):
  return text.lower()

In [46]:
data["inputs"]=data["inputs"].apply(clean)

In [47]:
data

,inputs,tags,label
0,hi,greeting,44
1,hey,greeting,44
2,is anyone there?,greeting,44
3,hi there,greeting,44
4,hello,greeting,44
...,...,...,...
227,how do i know if i'm unwell?,fact-29,32
228,how can i maintain social connections? what if i feel lonely?,fact-30,34
229,what's the difference between anxiety and stress?,fact-31,35
230,what's the difference between sadness and depression?,fact-32,36


In [48]:
import tensorflow
from tensorflow.keras.utils import to_categorical
import numpy as np
np.set_printoptions(threshold=np.inf)

y_train = to_categorical(data.label)

In [49]:
y_train.shape#


(232, 80)

In [50]:
y_train

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [51]:
!pip install transformers
import transformers


You should consider upgrading via the 'C:\Users\km_te\OneDrive\Documents\OmdenaProjects\chatbot-deployment-main\chatbot-deployment-main\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [52]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert = TFBertModel.from_pretrained("bert-base-cased")

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [53]:
x_train = tokenizer(
    text=data.inputs.tolist(),
    add_special_tokens=True,
    max_length=20,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [54]:
input_ids=x_train["input_ids"]

In [55]:
input_ids.shape

TensorShape([232, 20])

In [56]:
attention_mask=x_train["attention_mask"]

In [57]:
attention_mask.shape

TensorShape([232, 20])

In [58]:
y_train.shape

(232, 80)

In [59]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [60]:
max_len = 20
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(80,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [61]:
optimizer =  tf.keras.optimizers.legacy.Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [62]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    epochs=50,
    batch_size=36)

Epoch 1/50


c:\Users\km_te\OneDrive\Documents\OmdenaProjects\chatbot-deployment-main\chatbot-deployment-main\venv\lib\site-packages\keras\backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


7/7 [==============================] - 42s 2s/step - loss: 4.3493 - balanced_accuracy: 0.0302
Epoch 2/50
7/7 [==============================] - 17s 2s/step - loss: 4.1290 - balanced_accuracy: 0.0733
Epoch 3/50
7/7 [==============================] - 17s 2s/step - loss: 3.9212 - balanced_accuracy: 0.1336
Epoch 4/50
7/7 [==============================] - 17s 2s/step - loss: 3.6395 - balanced_accuracy: 0.2112
Epoch 5/50
7/7 [==============================] - 17s 2s/step - loss: 3.4001 - balanced_accuracy: 0.2414
Epoch 6/50
7/7 [==============================] - 17s 2s/step - loss: 3.1394 - balanced_accuracy: 0.2672
Epoch 7/50
7/7 [==============================] - 17s 2s/step - loss: 2.8499 - balanced_accuracy: 0.3750
Epoch 8/50
7/7 [==============================] - 17s 2s/step - loss: 2.6263 - balanced_accuracy: 0.4526
Epoch 9/50
7/7 [==============================] - 18s 2s/step - loss: 2.4184 - balanced_accuracy: 0.4914
Epoch 10/50
7/7 [==============================] - 18s 2s/step - l

In [63]:
texts = input(str('input the text'))
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=20,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100


1/1 [==============================] - 3s 3s/step


In [64]:
output=validation.argmax()

In [65]:
response_tag = le.inverse_transform([output])[0]

In [66]:
response_tag

'greeting'

In [67]:
import random

In [68]:
print("doctor Omdena: ",random.choice(responses[response_tag]))

doctor Omdena:  Hi there. How are you feeling today?


In [69]:
#chatting
import string
import random
while True:
  text_p=[]
  texts = input("You:")
  texts = [letters.lower() for letters in texts if letters not in string.punctuation]
  texts = ''.join(texts)
  text_p.append(texts)


  x_val = tokenizer(
  text=text_p,
  add_special_tokens=True,
  max_length=20,
  truncation=True,
  padding='max_length', 
  return_tensors='tf',
  return_token_type_ids = False,
  return_attention_mask = True,
  verbose = True) 
  validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
  output=validation.argmax()
  response_tag = le.inverse_transform([output])[0]
  print("doctor Omdena: ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break



1/1 [==============================] - 0s 74ms/step
doctor Omdena:  Can you elaborate on that?
1/1 [==============================] - 0s 73ms/step
doctor Omdena:  Bye! Come back again.


In [70]:
model.save("omdenatransformer.h5")

In [71]:

#installing the transformers library
!pip install transformers

You should consider upgrading via the 'C:\Users\km_te\OneDrive\Documents\OmdenaProjects\chatbot-deployment-main\chatbot-deployment-main\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [72]:

#importing the pretrained biobert tokenizer and the biobert model
from transformers import AutoTokenizer, TFAutoModel
biobert_tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/BioRedditBERT-uncased")

biobert_model = TFAutoModel.from_pretrained("cambridgeltl/BioRedditBERT-uncased")

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at cambridgeltl/BioRedditBERT-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [73]:
x_train = tokenizer(
    text=data.inputs.tolist(),
    add_special_tokens=True,
    max_length=20,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [74]:
import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(le, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)